# Contact of an elastic membrane with a rigid indenter: an augmented-Lagrangian approach

In [1]:
from dolfin import *
import numpy as np
import matplotlib.pyplot as plt
parameters["form_compiler"]["representation"] = 'quadrature'
import warnings
from ffc.quadrature.deprecation import QuadratureRepresentationDeprecationWarning
warnings.simplefilter("once", QuadratureRepresentationDeprecationWarning)

ppos = lambda x: (x-abs(x))/2.
Heav = lambda x: (1-x/abs(x))/2.

N = 40
mesh = UnitSquareMesh(N, N, "crossed")

obstacle = Expression("y+pow(x[0]-0.5,2)+pow(x[1]-0.5, 2)", y=-0.001, degree=2)

alpha = 100.
Nitermax = 10

V = FunctionSpace(mesh, "CG", 2)
#P = FunctionSpace(mesh, "Quadrature", quadrature_scheme="default")
Pe = FiniteElement("Quadrature", mesh.ufl_cell(), degree=2, quad_scheme='default')
P = FunctionSpace(mesh, Pe)

u = Function(V)
u_old = Function(V)
du = TrialFunction(V)
u_ = TestFunction(V)
h = Function(V)

p = Function(P)
p_old = Function(P)
gap = Function(P)

corr = Constant(0.)
rho = Function(P)
dxm = dx(metadata={"quadrature_degree": 2, "quadrature_scheme": "default"})
lagrangian = 0.5*inner(grad(u), grad(u))*dx + p*ppos(h-u)*dx + alpha/2.*ppos(h-u)**2*dx
form = inner(grad(u), grad(u_))*dx - p*Heav(h-u)*u_*dxm
J = derivative(form, u, du)

def local_project(v, V, u=None):
    dv = TrialFunction(V)
    v_ = TestFunction(V)
    a_proj = inner(dv, v_)*dxm
    b_proj = inner(v, v_)*dxm
    solver = LocalSolver(a_proj, b_proj)
    solver.factorize()
    if u is None:
        u = Function(V)
        solver.solve_local_rhs(u)
        return u
    else:
        solver.solve_local_rhs(u)
        return

def border(x, on_boundary):
    return on_boundary
bc = DirichletBC(V, Constant(0), border)

def solve_Uzawa(tol=2e-6):
    t = 1.
    for i in range(Nitermax):
        u_old.assign(u)
        p_old.assign(p)
        told = t

        local_project(p + alpha*ppos(h-u), P, p)
        solve(form == 0, u, bc, J=J)
        #gap.assign(project(h-u, P))
        #p.vector()[:] = np.minimum(0*p_old.vector().get_local(), rho.vector().get_local() + 
        #                           alpha*gap.vector().get_local())
        #direction = assemble((h-u)*(p-p_old)*dx)
        #t = (1+sqrt(1+4*told**2))/2.
        #if direction >= 0:
        #    corr=(t-1.)/told
        #else:
        #    print "    Restart..."
        #    corr = 0
        #    t = 1.
        #rho.vector()[:] = (1+corr)*p.vector().get_local() - corr*p_old.vector().get_local()
        res = max(0*errornorm(u, u_old), assemble((p- p_old)**2*dxm))
        print "Residual:", res
        if res < tol:
            break

h.interpolate(obstacle)     
solve_Uzawa()


/usr/lib/python2.7/dist-packages/ffc/jitcompiler.py:235: QuadratureRepresentationDeprecationWarning: 
*** ===================================================== ***
*** FFC: quadrature representation is deprecated! It will ***
*** likely be removed in 2018.1.0 release. Use uflacs     ***
*** representation instead.                               ***
*** ===================================================== ***
  issue_deprecation_warning()


Residual: 1.04457776733e-05
Residual: 8.09266832477e-06
Residual: 6.445405364e-06


RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to solve nonlinear system with NewtonSolver.
*** Reason:  Newton solver did not converge because maximum number of iterations reached.
*** Where:   This error was encountered inside NewtonSolver.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2017.2.0
*** Git changeset:  unknown
*** -------------------------------------------------------------------------


In [ ]:
pl = plot(u)
plt.colorbar(pl)
plt.show()

pl = plot(local_project(p, V))
plt.colorbar(pl)
plt.show()

In [14]:
for d in np.linspace(1e-4, 1e-3, 11):
    obstacle.y = d
    h.interpolate(obstacle)
    solve_Uzawa()

RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to solve nonlinear system with NewtonSolver.
*** Reason:  Newton solver did not converge because maximum number of iterations reached.
*** Where:   This error was encountered inside NewtonSolver.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2017.2.0
*** Git changeset:  unknown
*** -------------------------------------------------------------------------


In [6]:
assemble(form)

    Calling FFC just-in-time (JIT) compiler, this may take some time.


<dolfin.cpp.la.Vector; proxy of <Swig Object of type 'std::shared_ptr< dolfin::Vector > *' at 0x7f401e7570c0> >